# Google Drive

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
!mkdir chess_data_kaggle

In [ ]:
!unzip -o -q "/content/drive/MyDrive/Chess_Pieces.yolov8-obb.zip" -d "/content/chess_data_kaggle"


#Split data train/val/test format .ymal

In [ ]:
!pip install roboflow

In [ ]:
from roboflow import Roboflow
rf = Roboflow(api_key="kynevUNJySWKdBOVUBNg")
project = rf.workspace("noah-fbbgs").project("chess-piece-detector-zsj1p")
version = project.version(4)
dataset = version.download("yolov8")


# merger data

In [ ]:
import os
import shutil
from glob import glob
import random

# Path to your datasets (edit if needed)
DS1 = "/content/Chess-piece-tutorial-1"
DS2 = "/content/chess_data_kaggle"

OUTPUT = "/content/combined_chess_corners"

os.makedirs(OUTPUT, exist_ok=True)


In [ ]:
def collect_pairs(root):
    images = []
    for folder in ["train", "valid", "test"]:
        img_dir = os.path.join(root, folder, "images")
        lbl_dir = os.path.join(root, folder, "labels")

        if not os.path.exists(img_dir):
            continue

        for img in glob(img_dir + "/*"):
            base = os.path.basename(img)
            name, ext = os.path.splitext(base)
            lbl = os.path.join(lbl_dir, name + ".txt")

            if os.path.exists(lbl):
                images.append((img, lbl))
    return images

pairs1 = collect_pairs(DS1)
pairs2 = collect_pairs(DS2)

print("Dataset 1 samples:", len(pairs1))
print("Dataset 2 samples:", len(pairs2))


In [ ]:
all_pairs = pairs1 + pairs2
random.shuffle(all_pairs)

print("TOTAL merged samples:", len(all_pairs))


In [ ]:
train_ratio = 0.75
val_ratio = 0.15
test_ratio = 0.10

N = len(all_pairs)
train_end = int(N * train_ratio)
val_end = int(N * (train_ratio + val_ratio))

train_pairs = all_pairs[:train_end]
val_pairs = all_pairs[train_end:val_end]
test_pairs = all_pairs[val_end:]

print(len(train_pairs), len(val_pairs), len(test_pairs))


In [ ]:
for subset in ["train", "val", "test"]:
    os.makedirs(f"{OUTPUT}/images/{subset}", exist_ok=True)
    os.makedirs(f"{OUTPUT}/labels/{subset}", exist_ok=True)


In [ ]:
def copy_pairs(pairs, subset):
    for img, lbl in pairs:
        base = os.path.basename(img)
        name = os.path.splitext(base)[0]

        shutil.copy(img, f"{OUTPUT}/images/{subset}/{base}")
        shutil.copy(lbl, f"{OUTPUT}/labels/{subset}/{name}.txt")

copy_pairs(train_pairs, "train")
copy_pairs(val_pairs, "val")
copy_pairs(test_pairs, "test")

print("Dataset merged successfully!")


In [ ]:
data_yaml = f"""
train: {OUTPUT}/images/train
val: {OUTPUT}/images/val
test: {OUTPUT}/images/test

names:
  - corner
nc: 1
"""

with open(f"{OUTPUT}/data.yaml", "w") as f:
    f.write(data_yaml)

print("Created:", f"{OUTPUT}/data.yaml")


#split

In [ ]:
import shutil
import os

def copy_dataset_from_drive(src_path, dst_path):
    # Remove old local folder if exists
    if os.path.exists(dst_path):
        shutil.rmtree(dst_path)

    print(f"Copying {src_path} → {dst_path} ...")
    shutil.copytree(src_path, dst_path)
    print("Copy complete.\n")

# Example: Adjust paths if your Drive is mounted in /content/drive/MyDrive/
copy_dataset_from_drive("/content/drive/MyDrive/2568_1/Chess Pieces", "/content/Chess Pieces")
copy_dataset_from_drive("/content/drive/MyDrive/2568_1/boardchess", "/content/boardchess")
copy_dataset_from_drive("/content/drive/MyDrive/2568_1/chess_label_dataset", "/content/chess_label_dataset")


In [ ]:
import os
import glob
import random
import shutil

def autosplit_dataset(dataset_root, img_exts=[".jpg", ".png", ".jpeg"]):

    if('dataset' in dataset_root):
       img_dir = os.path.join(dataset_root,"Images")
       label_dir = os.path.join(dataset_root, "Label")
    else :
      img_dir = os.path.join(dataset_root, "train", "images")
      label_dir = os.path.join(dataset_root, "train", "labels")

    if not os.path.exists(img_dir):
        raise RuntimeError(f"Images folder not found: {img_dir}")
    if not os.path.exists(label_dir):
        raise RuntimeError(f"Labels folder not found: {label_dir}")

    for split in ["train", "valid", "test"]:
        os.makedirs(os.path.join(dataset_root, split, "images"), exist_ok=True)
        os.makedirs(os.path.join(dataset_root, split, "labels"), exist_ok=True)

    images = []
    for ext in img_exts:
        images.extend(glob.glob(os.path.join(img_dir, f"*{ext}")))

    if len(images) == 0:
        raise RuntimeError("No images found in dataset.")

    random.shuffle(images)

    n = len(images)
    n_train = int(0.7 * n)
    n_valid = int(0.2 * n)
    n_test = n - n_train - n_valid

    train_imgs = images[:n_train]
    valid_imgs = images[n_train:n_train+n_valid]
    test_imgs = images[n_train+n_valid:]

    def move_files(image_list, split_name):
        for img_path in image_list:
            fname = os.path.basename(img_path)
            lbl = os.path.splitext(fname)[0] + ".txt"
            label_path = os.path.join(label_dir, lbl)

            shutil.move(img_path, os.path.join(dataset_root, split_name, "images", fname))

            if os.path.exists(label_path):
                shutil.move(label_path, os.path.join(dataset_root, split_name, "labels", lbl))

    move_files(train_imgs, "train")
    move_files(valid_imgs, "valid")
    move_files(test_imgs, "test")

    print("=== Dataset Split Completed ===")
    print("Total:", n)
    print("Train:", len(train_imgs))
    print("Valid:", len(valid_imgs))
    print("Test :", len(test_imgs))
    print("Output in:", dataset_root)

autosplit_dataset("/content/Chess Pieces")
autosplit_dataset("/content/boardchess")
autosplit_dataset("/content/chess_label_dataset")

# model

In [ ]:
!pip install ultralytics

In [ ]:
import ultralytics

from ultralytics import YOLO
import os
from PIL import Image
import cv2
from IPython.display import Video
import glob
import matplotlib.pyplot as plt
import numpy as np
import warnings
warnings.filterwarnings('ignore')


ultralytics.checks()

In [ ]:
!rm -rf /content/chess_label_dataset/Images
!rm -rf /content/chess_label_dataset/Label

In [ ]:
!cp /content/boardchess/data.yaml /content/chess_label_dataset

#Detect Piece

In [ ]:
!grep -R "^13 " "/content/chess_label_dataset/test/labels"


In [ ]:
from ultralytics import YOLO

model = YOLO('yolov8m.pt')
model.to('cuda')

results = model.train(
    data='/content/Chess Pieces/data.yaml',
    epochs=20,
    patience=3,
    batch=16,
    lr0=0.0001,
    lrf=0.1,
    imgsz=640,
    plots= True,
    cache=False,
    device=0
)

In [ ]:
metrics = model.val()  # optionally: model.val(data="your_data.yaml")
print(metrics.box.map50)     # mAP50
print(metrics.box.map)       # mAP50-95
print(metrics.box.precision) # precision
print(metrics.box.recall)    # recall

# Result

In [ ]:
model = YOLO('/content/runs/detect/train2/weights/best.pt')

In [ ]:
model

In [ ]:
example_video_path = '/content/Chess Detection Competition/test_videos/2_Move_rotate_student.mp4'

video_output = model.predict(
    source=example_video_path,
    conf=0.6,
    save=True,
    device=0   # <-- force GPU
)


In [ ]:
results = model("/content/Screenshot 2025-12-11 135751.png")

predicted_classes = [model.names[int(box.cls)] for box in results[0].boxes]
print(predicted_classes)

In [ ]:
image_test_path = '/content/Screenshot 2025-12-11 135751.png'

results = model.predict(source=image_test_path,
                        imgsz=640)

test_image = results[0].plot(line_width=2)
plt.imshow(test_image)

In [ ]:
image_test_path = '/content/Screenshot 2025-12-11 135751.png'

results = model.predict(source=image_test_path,
                        imgsz=640)

test_image = results[0].plot(line_width=2)
plt.imshow(test_image)

In [ ]:
image_test_path = '/content/frame_000000.png'

results = model.predict(source=image_test_path,
                        imgsz=640)

test_image = results[0].plot(line_width=2)
plt.imshow(test_image)

In [ ]:
image_test_path = '/content/frame_000038.png'

results = model.predict(source=image_test_path,
                        imgsz=640)

test_image = results[0].plot(line_width=2)
plt.imshow(test_image)